In [1]:
import requests
import bs4
import pandas as pd
import json
import time
import random
import numpy as np
from tqdm import tqdm
from datetime import date
import glob
import datetime

# Settings

In [2]:
no_repeat_watch_days = 10
n_post_target = 50
EMAIL = 'cattaiangela2018@gmail.com'
PASSWORD = 'dontbotherme'

def cal_engagement(replies, reblogs, likes, followers):
    eng_rate = (likes+replies*3+reblogs*2)/followers
    return eng_rate

date_format = '%Y-%m-%d'
today = date.today()
date_str = '_'+str(today).replace('-','')
user_agent = [
    "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.114 Safari/537.36",
    "Mozilla/4.0 (compatible; MSIE 7.0; Windows NT 6.0; Acoo Browser; SLCC1; .NET CLR 2.0.50727; Media Center PC 5.0; .NET CLR 3.0.04506)",
    "Mozilla/4.0 (compatible; MSIE 7.0; AOL 9.5; AOLBuild 4337.35; Windows NT 5.1; .NET CLR 1.1.4322; .NET CLR 2.0.50727)",
    "Mozilla/5.0 (Windows; U; MSIE 9.0; Windows NT 9.0; en-US)",
    "Mozilla/5.0 (compatible; MSIE 9.0; Windows NT 6.1; Win64; x64; Trident/5.0; .NET CLR 3.5.30729; .NET CLR 3.0.30729; .NET CLR 2.0.50727; Media Center PC 6.0)",
    "Mozilla/5.0 (compatible; MSIE 8.0; Windows NT 6.0; Trident/4.0; WOW64; Trident/4.0; SLCC2; .NET CLR 2.0.50727; .NET CLR 3.5.30729; .NET CLR 3.0.30729; .NET CLR 1.0.3705; .NET CLR 1.1.4322)",
    "Mozilla/4.0 (compatible; MSIE 7.0b; Windows NT 5.2; .NET CLR 1.1.4322; .NET CLR 2.0.50727; InfoPath.2; .NET CLR 3.0.04506.30)",
    "Mozilla/5.0 (Windows; U; Windows NT 5.1; zh-CN) AppleWebKit/523.15 (KHTML, like Gecko, Safari/419.3) Arora/0.3 (Change: 287 c9dfb30)",
    "Mozilla/5.0 (X11; U; Linux; en-US) AppleWebKit/527+ (KHTML, like Gecko, Safari/419.3) Arora/0.6",
    "Mozilla/5.0 (Windows; U; Windows NT 5.1; en-US; rv:1.8.1.2pre) Gecko/20070215 K-Ninja/2.1.1",
    "Mozilla/5.0 (Windows; U; Windows NT 5.1; zh-CN; rv:1.9) Gecko/20080705 Firefox/3.0 Kapiko/3.0",
    "Mozilla/5.0 (X11; Linux i686; U;) Gecko/20070322 Kazehakase/0.4.5",
    "Mozilla/5.0 (X11; U; Linux i686; en-US; rv:1.9.0.8) Gecko Fedora/1.9.0.8-1.fc10 Kazehakase/0.5.6",
    "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/535.11 (KHTML, like Gecko) Chrome/17.0.963.56 Safari/535.11",
    "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_7_3) AppleWebKit/535.20 (KHTML, like Gecko) Chrome/19.0.1036.7 Safari/535.20"
]

url_base = 'https://gab.com/'
ua = random.choice(user_agent)

# Functions

In [3]:
def write_json(dict_all, file_name):
    path = 'user_data/'
    with open(path+file_name+'.json', 'w') as f:
        json.dump(dict_all, f)
def get_basic_info(res_post):
    post = res_post[0]
    dict_user = dict()
    dict_user['user_id'] = post['account']['id']
    dict_user['date_joined'] = post['account']['created_at']
    dict_user['user_name'] = post['account']['username']
    dict_user['gabs'] = post['account']['statuses_count']
    dict_user['followers'] = post['account']['followers_count']
    dict_user['following'] = post['account']['following_count']
    return dict_user

In [4]:
def get_n_posts(dict_user, n_post_target,user_id, rs, headers):
    n_post = len(dict_user['last_50_posts'])
    last_id = dict_user['last_50_posts'][-1]['post_id']

    url_base = 'https://gab.com/'
    while (n_post < n_post_target):
        try:
            time.sleep(random.randint(3,10))
            uri = 'api/v1/accounts/'+user_id+'/statuses?exclude_replies=true&max_id='+last_id
            url2 = url_base+uri
            response = rs.get(url2, headers = headers)
            res = json.loads(response.text)
        except Exception as e:
            print('user_id = ', user_id)
            print('[get_n_posts] Failed when n_post = ', n_post)
            print(e)
        if (len(response.text) < 5)|(len(res) < 5):
            break
        else:
            last_id = res[-1]['id']
            res_posts = [get_post_dict(post) for post in res]
            dict_user['last_50_posts'].extend(res_posts)
            n_post = len(dict_user['last_50_posts'])
    return dict_user

In [7]:
def get_post_dict(post):
    dict_post = dict()
    dict_post['post_id'] = post['id']
    if post['reblog'] is None:
        dict_post['is_repost'] = 'no'
        dict_post['replies_count'] = post['replies_count']
        dict_post['reblogs_count'] = post['reblogs_count']
        dict_post['favourites_count'] = post['favourites_count']
        dict_post['content'] = post['content']
    else:
        dict_post['is_repost'] = 'yes'
        dict_post['replies_count'] = post['reblog']['replies_count']
        dict_post['reblogs_count'] = post['reblog']['reblogs_count']
        dict_post['favourites_count'] = post['reblog']['favourites_count']
        dict_post['reblog_content'] = post['reblog']['content']
        dict_post['reblog_account'] = post['reblog']['account']
        dict_post['reblog_media_attachments'] = post['reblog']['media_attachments']
    dict_post['media_attachments'] = post['media_attachments']
    dict_post['tags'] = post['tags']
    return dict_post

In [8]:
def get_user_data(user_id,rs,headers):
    url = 'https://gab.com/api/v1/accounts/'+user_id+'/statuses?exclude_replies=true'
    html = rs.get(url, headers=headers)
    obj = bs4.BeautifulSoup(html.text, 'lxml')
    text = obj.select('p')[0].text
    res_post = json.loads(text)
    # get user data and posts on first page
    dict_user = get_basic_info(res_post)
    last_50_posts = [get_post_dict(post) for post in res_post]
    dict_user['last_50_posts'] = last_50_posts
    
    # get rest posts
    dict_user = get_n_posts(dict_user, n_post_target,user_id, rs, headers)
    # avg_engagement
    f = dict_user['followers']
    eng_rates = [cal_engagement(x['replies_count'], x['reblogs_count'], x['favourites_count'],f) for x in dict_user['last_50_posts'] if x['is_repost'] == 'no']
    dict_user['avg_engagement'] = np.mean(eng_rates)
    write_json(dict_user, user_id)
    return dict_user

In [9]:
def parse_explore(response):
    obj = bs4.BeautifulSoup(response.text, 'lxml')
    text = obj.select('p')[0].text
    explore_post = json.loads(text)
    return explore_post

# login

In [10]:
# login page & get token
LOGIN_URL = 'https://gab.com/auth/sign_in'
rs = requests.session()

result = rs.get(LOGIN_URL)
obj = bs4.BeautifulSoup(result.text, 'lxml')
token = str(obj.find('input',attrs={'name':"authenticity_token"}))
authenticity_token = token.split('"')[-2]
cookie = rs.cookies.get_dict()
cookie = list(cookie.keys())[0]+'='+list(cookie.values())[0]+';'+list(cookie.keys())[1]+'='+list(cookie.values())[1]

In [11]:
# login & get cookie
headers = {
    'Connection': 'keep-alive',
    'Origin': 'https://gab.com',
    'User-Agent': ua,
    'Referer': LOGIN_URL,
    'Cookie': cookie
}

data = {'user[email]':EMAIL,'user[password]':PASSWORD,'authenticity_token':authenticity_token}
time.sleep(3)
rs = requests.session()
log_in = rs.post('https://gab.com/auth/sign_in', headers = headers, data = data)
cookie = rs.cookies.get_dict()
cookie = list(cookie.keys())[0]+'='+list(cookie.values())[0]+';'+list(cookie.keys())[1]+'='+list(cookie.values())[1]+';'+list(cookie.keys())[2]+'='+list(cookie.values())[2]

# first: get target users

## get target_lists from:
- explore hot stories...
- and compare with our watch_user_list already monitoring

In [12]:
# get data from explore page
headers = {
    'Connection': 'keep-alive',
    'User-Agent': ua,
    'Referer': url_base,
    'Cookie':cookie
}

url = 'https://gab.com/api/v1/timelines/explore?page=1&sort_by=hot'
response = rs.get(url,headers = headers)
explore_post = parse_explore(response)

In [13]:
for i in tqdm(range(2,20)):
    try:
        time.sleep(random.randint(3,8))
        url = 'https://gab.com/api/v1/timelines/explore?page='+str(i)+'&sort_by=hot'
        response = rs.get(url,headers = headers)
        add_explore_post = parse_explore(response)
        explore_post.extend(add_explore_post)
    except Exception as e:
        print(e)
    if len(str(response.text)) < 5:
        print('Completed: explore data ends from page:', i)
        break
target_users = list(set([post['account']['id'] for post in explore_post]))
write_json(explore_post, 'explore_post'+date_str)

 39%|███▉      | 7/18 [01:01<01:35,  8.71s/it]

Completed: explore data ends from page: 9


# second: get posts of target users 

In [14]:
# compare watching list
# we can compare then with data from time difference(timedelta) less then let's say 10 days 
# (which means in 10 days we don't update same user's data)
try:
    with open('user_data/watch_user_list.json') as f:
        watch_list = json.load(f)
except:
    watch_list = dict()

watch_dates = list(watch_list.keys())
days = datetime.timedelta(no_repeat_watch_days)
old_date = today - days
compare_dates = [d for d in watch_dates if (str(old_date)<=d)&(d<=str(today))]
already_watch = []
[already_watch.extend(watch_list[d]) for d in compare_dates]
already_watch = list(set(already_watch))

new_targets = [x for x in target_users if x not in already_watch]

In [15]:
headers = {
    'Connection': 'keep-alive',
    'User-Agent': ua,
    'Referer': url_base,
    'Accept':'application/json, text/plain, */*',
#     'Accept-Encoding':'gzip, deflate, br',
#     'Accept-Language':'en-US,en;q=0.9',
    'Cookie':cookie
}

# get all user data
dict_all_user = dict()
path = 'user_data/'
files = glob.glob(path+"*")
new_name = path+'all_user_data'+date_str+'.json'
try:
    for uid in tqdm(new_targets):
        dict_all_user[uid] = get_user_data(uid, rs, headers) 
except Exception as e:
    print(e)
    if new_name in files:
        t = time.localtime()
        current_time = time.strftime("%H:%M:%S", t)[:-3].replace(':', '')
        write_json(dict_all_user, 'all_user_data'+date_str+'_'+current_time)
    else:
        write_json(dict_all_user, 'all_user_data'+date_str)

if new_name in files:
    t = time.localtime()
    current_time = time.strftime("%H:%M:%S", t)[:-3].replace(':', '')
    write_json(dict_all_user, 'all_user_data'+date_str+'_'+current_time)
else:
    write_json(dict_all_user, 'all_user_data'+date_str)

 20%|██        | 1/5 [00:17<01:10, 17.73s/it]

0


In [16]:
# add new users into new watch_list
try:
    watch_list[str(date.today())].extend(target_users)
except:
    watch_list[str(date.today())] = target_users
    
watch_list[str(date.today())] = list(set(watch_list[str(date.today())]))
write_json(watch_list, 'watch_user_list')